In [1]:
import numpy as np

Xdata = np.load("../data/6-parameter-maps/X_maps_Cosmogrid_100k.npy", allow_pickle=True)
Ydata = np.load("../data/6-parameter-maps/y_maps_Cosmogrid_100k.npy", allow_pickle=True)

num_samples = len(Xdata)
train_split, val_split, test_split = int(0.80*num_samples), \
            int(0.10*num_samples), int(0.10*num_samples) + 1

train_x, val_x, test_x = np.split(Xdata[:, :, :, None], [train_split, train_split+val_split])
train_y, val_y, test_y = np.split(Ydata, [train_split, train_split+val_split])

output_num = 5

train_y, val_y, test_y = train_y[:,:output_num], val_y[:,:output_num], test_y[:,:output_num]

print(f"x_train shape: {train_x.shape} - y_train shape: {train_y.shape}")
print(f"x_test shape: {test_x.shape} - y_test shape: {test_y.shape}")

x_train shape: (80000, 66, 66, 1) - y_train shape: (80000, 5)
x_test shape: (10000, 66, 66, 1) - y_test shape: (10000, 5)


In [2]:
# create jsonl files from these train, val, test splits
import jsonlines

for split_name, x, y in zip(['train', 'val', 'test'], [train_x, val_x, test_x], [train_y, val_y, test_y]):
    print(split_name)
    with jsonlines.open(f'../data/6-parameter-maps/{split_name}.jsonl', mode='w') as writer:
        for i in range(len(x)):
            Omega_m, H_0, n_s, sigma_8, w_0 = y[i].tolist()
            if list(map(type, [Omega_m, H_0, n_s, sigma_8, w_0])) != [float]*5:
                print(Omega_m, H_0, n_s, sigma_8, w_0)
            writer.write({'map': x[i].tolist(), 
                            'Omega_m': float(Omega_m), 'H_0': float(H_0), 'n_s': float(n_s), 
                            'sigma_8': float(sigma_8), 'w_0': float(w_0)})
            # break
    # break


train
val
test
